<a href="https://colab.research.google.com/github/springboardmentor891v/Email_Assistant_Using_LangGraph/blob/manoj_preetham/Email_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langgraph langchain langchain-openai google-auth-oauthlib google-api-python-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 4.2 MB/s eta 0:00:00


In [2]:
import os
import base64
from typing import TypedDict, Annotated, List, Literal
from email.mime.text import MIMEText

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Google API Imports
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

In [3]:
# 1. SETUP GMAIL AUTHENTICATION
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file(
            'token.json', SCOPES
        )

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credential.json',
                SCOPES,
                redirect_uri='urn:ietf:wg:oauth:2.0:oob'
            )

            auth_url, _ = flow.authorization_url(
                prompt='consent'
            )

            print("\n🔗 OPEN THIS URL IN YOUR BROWSER:\n")
            print(auth_url)

            code = input("\n📌 Paste the authorization code here: ")
            flow.fetch_token(code=code)
            creds = flow.credentials

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)

service = get_gmail_service()


🔗 OPEN THIS URL IN YOUR BROWSER:

https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=915502763627-c2ilmm6t0n042r0lbjg7mjr0f0mmqehn.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=62Tx1TVi8ZhOoiTyzCD8XU58ORmHoU&prompt=consent&access_type=offline

📌 Paste the authorization code here: 4/1ASc3gC1qNxu3zF67nJyuX2E1tHtfwM4nt6aP4teWzsy5rbR6nmSgG6RFWuo


In [4]:
import os
from openai import OpenAI
from google.colab import userdata

# =========================
# OPENAI SETUP
# =========================
api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in Colab secrets")

client = OpenAI(api_key=api_key)

def llm_call(prompt: str):
    r = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return r.choices[0].message.content.strip()

In [5]:
import os
from openai import OpenAI

# Google Colab specific import for secrets
from google.colab import userdata

# Read API key from Colab secrets
api_key = userdata.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in Colab secrets. Please ensure it's set in the Colab 'Secrets' tab.")

# Create OpenAI client
client = OpenAI(api_key=api_key)

try:
    # List models available for this key
    models = client.models.list()

    print("Models available for this key:")
    for model in models.data:
        print(model.id)

except Exception as e:
    print("Error:", e)


Models available for this key:
gpt-3.5-turbo
gpt-5.2-codex
gpt-4o-mini-tts-2025-12-15
gpt-realtime-mini-2025-12-15
gpt-audio-mini-2025-12-15
chatgpt-image-latest
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-3.5-turbo-0125
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
gpt-4o-audio-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-audio-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-search-preview
gpt-4o-transcribe
gpt-4o-mini-transcribe
gpt-4o-mini-tts
o3-2025-04-16
o4-mini-2025-04-16
o3
o4-mini
gpt-4.1-2025-04-14
gpt-4.1
gpt-4.1-mini-2025-04-14
gpt-4.1-mini
gpt-4.1-nano-2025-04-14
gpt-4.1-nano

In [6]:
# =========================
# READ EMAIL
# =========================
def extract_email_data(service, msg_id):
    message = service.users().messages().get(
        userId='me',
        id=msg_id,
        format='full'
    ).execute()

    headers = message['payload']['headers']
    subject = sender = date = ""

    for h in headers:
        if h['name'] == 'Subject':
            subject = h['value']
        elif h['name'] == 'From':
            sender = h['value']
        elif h['name'] == 'Date':
            date = h['value']

    body = ""
    payload = message['payload']

    if 'parts' in payload:
        for part in payload['parts']:
            if part['mimeType'] == 'text/plain':
                data = part['body'].get('data')
                if data:
                    body = base64.urlsafe_b64decode(data).decode('utf-8')
    else:
        data = payload['body'].get('data')
        if data:
            body = base64.urlsafe_b64decode(data).decode('utf-8')

    return subject, sender, date, body
# =========================
# FETCH UNREAD
# =========================
results = service.users().messages().list(
    userId='me',
    labelIds=['UNREAD'],
    maxResults=100
).execute()

messages = results.get('messages', [])
print(f"📬 Unread emails found: {len(messages)}")
# =========================
# PREVIEW (YOUR STYLE)
# =========================
emails = []
for msg in messages:
    msg_id = msg['id']
    thread_id = msg['threadId']
    subject, sender, date, body = extract_email_data(service, msg_id)

    print("📧 SUBJECT:", subject)
    print("👤 FROM:", sender)
    print("📅 DATE:", date)
    print("📝 BODY (preview):", body[:300])
    print("-" * 60)

    emails.append((msg_id, thread_id, subject, sender, body))

📬 Unread emails found: 3
📧 SUBJECT: Security alert
👤 FROM: Google <no-reply@accounts.google.com>
📅 DATE: Sun, 01 Feb 2026 11:44:12 GMT
📝 BODY (preview): [image: Google]
You allowed Email-Assistant access to some of your Google Account data


manojpreetham200524@gmail.com

If you didn't allow Email-Assistant access to some of your Google Account
data, someone else may be trying to access your Google Account data.

Take a moment now to check 
------------------------------------------------------------
📧 SUBJECT: Fwd: Document verification
👤 FROM: Shane Gilbert S <shanegilbert1126@gmail.com>
📅 DATE: Sat, 24 Jan 2026 11:36:46 +0530
📝 BODY (preview): ---------- Forwarded message ---------
From: Shane Gilbert S <shanegilbert1126@gmail.com>
Date: Thu, 22 Jan 2026, 9:08 am
Subject: Document verification
To: manojpreetham200524@gmail.com <manojpreetham200524@gmail.com>


hey manoj,this is regarding document verification for your educational

----------------------------------------------------

In [9]:
import time
from openai import RateLimitError

# =====================================================
# CONFIGURATION (RATE LIMIT SAFE)
# =====================================================
REQUEST_DELAY = 1.2          # seconds between API calls (~50 RPM)
MAX_SUBJECT_CHARS = 300
MAX_BODY_CHARS = 3000

VALID_CATEGORIES = [
    "Emergency", "Work", "Promotion",
    "Spam", "Not-Useful", "Personal"
]

classification_cache = {}

# =====================================================
# TEXT SAFETY (TOKEN CONTROL)
# =====================================================
def trim_text(text, max_chars):
    if not text:
        return ""
    return text[:max_chars]

# =====================================================
# RULE-BASED FALLBACK (NO API COST)
# =====================================================
def rule_based_classifier(subject, body):
    text = (subject + " " + body).lower()

    if any(w in text for w in ["urgent", "asap", "immediately", "hospital"]):
        return "Emergency"
    if any(w in text for w in ["meeting", "project", "deadline", "manager"]):
        return "Work"
    if any(w in text for w in ["offer", "sale", "discount", "deal"]):
        return "Promotion"
    if any(w in text for w in ["lottery", "win money", "free prize"]):
        return "Spam"
    if any(w in text for w in ["family", "friend", "personal"]):
        return "Personal"

    return "Not-Useful"

# =====================================================
# SAFE EMAIL CLASSIFIER (RATE-LIMIT PROTECTED)
# =====================================================
def classify_email(subject, body):
    subject = trim_text(subject, MAX_SUBJECT_CHARS)
    body = trim_text(body, MAX_BODY_CHARS)

    cache_key = subject + body
    if cache_key in classification_cache:
        return classification_cache[cache_key]

    prompt = f"""
    Classify the following email into one of these categories:
    Emergency, Work, Promotion, Spam, Not-Useful, Personal.

    Return ONLY the category name.

    Email Subject: {subject}
    Email Body: {body}
    """

    try:
        time.sleep(REQUEST_DELAY)  # 🚦 RPM protection
        category = llm_call(prompt).strip()

        for vc in VALID_CATEGORIES:
            if vc.lower() in category.lower():
                classification_cache[cache_key] = vc
                return vc

        classification_cache[cache_key] = "Not-Useful"
        return "Not-Useful"

    except RateLimitError:
        fallback = rule_based_classifier(subject, body)
        classification_cache[cache_key] = fallback
        return fallback

    except Exception:
        return "Not-Useful"

# =====================================================
# FETCH UNREAD EMAILS (GMAIL API)
# =====================================================
results = service.users().messages().list(
    userId='me',
    labelIds=['UNREAD'],
    maxResults=100   # ⛔ SAFE LIMIT (increase slowly)
).execute()

messages = results.get('messages', [])

# =====================================================
# SUMMARY STATS
# =====================================================
stats = {
    "Emergency": 0,
    "Work": 0,
    "Promotion": 0,
    "Spam": 0,
    "Not-Useful": 0,
    "Personal": 0
}

preview_cache = []

# =====================================================
# MAIN LOOP (SAFE CLASSIFICATION)
# =====================================================
for msg in messages:
    msg_id = msg['id']

    # Must return: subject, sender, date, body
    subject, sender, date, body = extract_email_data(service, msg_id)

    category = classify_email(subject, body)

    stats[category] += 1
    preview_cache.append(
        (msg_id, msg.get("threadId"), subject, sender, body, category)
    )

# =====================================================
# OUTPUT
# =====================================================
print("\n📬 UNREAD EMAIL SUMMARY")
print("Total:", len(messages))
print("🚨 Emergency:", stats["Emergency"])
print("💼 Work:", stats["Work"])
print("📢 Promotion:", stats["Promotion"])
print("🗑 Spam:", stats["Spam"])
print("❌ Not Useful:", stats["Not-Useful"])
print("👤 Personal:", stats["Personal"])



📬 UNREAD EMAIL SUMMARY
Total: 3
🚨 Emergency: 1
💼 Work: 0
📢 Promotion: 0
🗑 Spam: 0
❌ Not Useful: 2
👤 Personal: 0


In [16]:
import time
import base64
from typing import TypedDict
from email.mime.text import MIMEText
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# =====================================================
# CONFIG
# =====================================================
YOUR_NAME = "Manoj Preetham V K"

MAX_SUBJECT_CHARS = 300
MAX_BODY_CHARS = 3000

# =====================================================
# UTILS
# =====================================================
def trim_text(text, limit):
    if not text:
        return ""
    return text[:limit]

def is_no_reply(body, sender):
    text = (body + " " + sender).lower()
    return any(x in text for x in ["do not reply", "no-reply", "noreply", "automated"])

# =====================================================
# LLM REPLY GENERATION (ONLY PLACE API IS USED)
# =====================================================
def generate_options(subject, sender, body):
    subject = trim_text(subject, MAX_SUBJECT_CHARS)
    body = trim_text(body, MAX_BODY_CHARS)

    prompt = f"""
You are an intelligent email assistant.

Email:
Subject: {subject}
From: {sender}
Body:
{body}

Generate THREE reply options.
Always sign every reply with this name: {YOUR_NAME}

Option 1:
Formal reply

Option 2:
Friendly reply

Option 3:
Short reply
"""

    try:
        return llm_call(prompt)

    except RateLimitError:
        # 🔒 HARD FALLBACK WHEN QUOTA IS EXHAUSTED
        return f"""
Option 1:
Thank you for the update. I will review this and get back to you shortly.

{YOUR_NAME}

Option 2:
Thanks for letting me know! I’ll take a look and respond soon 🙂

{YOUR_NAME}

Option 3:
Got it, thanks!

{YOUR_NAME}
"""

    except Exception:
        return f"""
Option 1:
Thank you.

{YOUR_NAME}
"""


# =====================================================
# GMAIL ACTION
# =====================================================
def send_reply(service, to, subject, reply_text, thread_id):
    msg = MIMEText(reply_text)
    msg["to"] = to
    msg["subject"] = "Re: " + subject

    raw = base64.urlsafe_b64encode(msg.as_bytes()).decode()

    service.users().messages().send(
        userId="me",
        body={"raw": raw, "threadId": thread_id}
    ).execute()

# =====================================================
# STATE
# =====================================================
class EmailState(TypedDict):
    msg_id: str
    thread_id: str
    subject: str
    sender: str
    body: str
    category: str        # ⬅ comes from FIRST CODE
    options: str
    decision: str
    selected_reply: str

# =====================================================
# GRAPH NODES (NO CLASSIFICATION HERE)
# =====================================================
def filter_node(state: EmailState):
    if state["category"] in ["Spam", "Not-Useful"]:
        print(f"🗑 Filtered: {state['subject']} ({state['category']})")
    return state

def generate_node(state: EmailState):
    options = generate_options(
        state["subject"],
        state["sender"],
        state["body"]
    )
    return {"options": options}

def hitl_node(state: EmailState):
    print("\n" + "=" * 70)
    print("📧 SUBJECT :", state["subject"])
    print("👤 FROM    :", state["sender"])
    print("🏷 CATEGORY:", state["category"])
    print("=" * 70)

    if is_no_reply(state["body"], state["sender"]):
        print("🚫 No-reply / automated email detected")
        return {"decision": "skip"}

    print(state["options"])

    choice = input("\nChoose (1/2/3), e=edit, s=skip: ").strip().lower()

    if choice == "s":
        return {"decision": "skip"}

    if choice == "e":
        text = input("\n✏️ Enter your reply:\n")
        if not text.strip().endswith(YOUR_NAME):
            text += f"\n\n{YOUR_NAME}"
        return {"decision": "send", "selected_reply": text}

    if choice not in ["1", "2", "3"]:
        return {"decision": "skip"}

    part = state["options"].split(f"Option {choice}:")
    if len(part) < 2:
        return {"decision": "skip"}

    reply = part[1].split("Option")[0].strip()
    return {"decision": "send", "selected_reply": reply}

def action_node(state: EmailState):
    if state["decision"] != "send":
        return

    send_reply(
        service,
        state["sender"],
        state["subject"],
        state["selected_reply"],
        state["thread_id"]
    )

    service.users().messages().modify(
        userId="me",
        id=state["msg_id"],
        body={"removeLabelIds": ["UNREAD"]}
    ).execute()

    print("📤 Reply sent & email marked as read")

# =====================================================
# GRAPH BUILD (STARTS FROM FILTER)
# =====================================================
graph = StateGraph(EmailState)

graph.add_node("filter", filter_node)
graph.add_node("generate", generate_node)
graph.add_node("hitl", hitl_node)
graph.add_node("action", action_node)

graph.set_entry_point("filter")

graph.add_conditional_edges(
    "filter",
    lambda s: END if s["category"] in ["Spam", "Not-Useful"] else "generate"
)

graph.add_edge("generate", "hitl")
graph.add_edge("hitl", "action")
graph.add_edge("action", END)

agent = graph.compile(checkpointer=MemorySaver())

# =====================================================
# RUN (USES CLASSIFIED OUTPUT FROM FIRST CODE)
# preview_cache = (msg_id, thread_id, subject, sender, body, category)
# =====================================================
for msg_id, thread_id, subject, sender, body, category in preview_cache:
    agent.invoke(
        {
            "msg_id": msg_id,
            "thread_id": thread_id,
            "subject": subject,
            "sender": sender,
            "body": body,
            "category": category,   # ✅ reused, NOT recomputed
        },
        config={"configurable": {"thread_id": thread_id}}
    )


🗑 Filtered: Security alert (Not-Useful)

📧 SUBJECT : Fwd: Document verification
👤 FROM    : Shane Gilbert S <shanegilbert1126@gmail.com>
🏷 CATEGORY: Emergency

Option 1:
Thank you for the update. I will review this and get back to you shortly.

Manoj Preetham V K

Option 2:
Thanks for letting me know! I’ll take a look and respond soon 🙂

Manoj Preetham V K

Option 3:
Got it, thanks!

Manoj Preetham V K


Choose (1/2/3), e=edit, s=skip: 1
📤 Reply sent & email marked as read
🗑 Filtered: hello iam Tony stark died as a hero and reborn as a villlian named victor von doom (Not-Useful)
